In [1]:
%load_ext autoreload
%autoreload 2

In [94]:
import pandas as pd
import pickle
import random
import collections
import re
import seaborn as sns
from dime_take_home.utils import get_locations_from_text

# Start to extract locations from news articles

In [64]:
with open("../data/raw/id_english_location_name.pkl", "rb") as f:
    locations_dict_en = pickle.load(f)
with open("../data/raw/id_arabic_location_name.pkl", "rb") as f:
    locations_dict_ar = pickle.load(f)

In [ ]:
with open("../data/processed/ar_locations_reversed.pkl", "rb") as f:
    locations_dict_ar_reversed = pickle.load(f)

with open("../data/processed/en_locations_reversed.pkl", "rb") as f:
    locations_dict_en_reversed = pickle.load(f)
    
# Grab reversed location dictionaries

### Define and test location regex

In [56]:
locations_pattern_en = re.compile(r"\b(?:%s)\b" % "|".join(map(re.escape, locations_dict_en_reversed.keys())),re.IGNORECASE)
locations_pattern_ar = re.compile(r"\b(?:%s)\b" % "|".join(map(re.escape, locations_dict_ar_reversed.keys())),re.IGNORECASE)
# Compile regex pattern for matching locations
# Important to add whitespace boundaries (\b) to avoid partial matches

In [ ]:
assert locations_pattern_en.findall('Suwayda') == ['Suwayda']
assert locations_pattern_en.findall('Suwayda mosul') == ['Suwayda','mosul']
assert locations_pattern_en.findall('inSuwayda') == []
assert locations_pattern_ar.findall(u'ذهبت إلى صيدا') == [u'صيدا']
assert locations_pattern_ar.findall(u'ذهبت إلى صيدا و طرابلس') == [u'صيدا', u'طرابلس']
# Check if regex patterns work as expected


In [6]:
df_news_en = pd.read_csv("../data/raw/news-articles-eng.csv")
df_news_en['dateTime'] = pd.to_datetime(df_news_en['dateTime'])
df_news_en.sort_values('dateTime', inplace=True)

df_news_ar = pd.read_csv("../data/raw/news-articles-ara.csv")
df_news_ar['dateTime'] = pd.to_datetime(df_news_ar['dateTime'])
df_news_ar.sort_values('dateTime', inplace=True)

# Grab news articles, convert dateTime to datetime object and sort by date

In [8]:
df_news_en.shape[0]

86660

In [22]:
if True:
    df_news_en = df_news_en.iloc[:df_news_en.shape[0]//50, :]
# Grab a subset 

In [23]:
df_news_en.shape

(1733, 20)

In [95]:
test_article = df_news_en.iloc[1, :]['body']

get_locations_from_text(test_article, locations_pattern_en, locations_dict_en_reversed)

['iq_an_6', 'iq_an_6', 'jo']

In [96]:
test_article = df_news_ar.iloc[2, :]['body']

get_locations_from_text(test_article, locations_pattern_ar, locations_dict_ar_reversed)

['iq_bg', 'iq_bg', 'iq_bg', 'iq_bg_10', 'iq_bg']

### Define and test risk factor regex

In [ ]:
df_risk_factors = pd.read_excel("../data/raw/risk-factors.xlsx")